In [1]:
pip install scipy

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install kaleido

Note: you may need to restart the kernel to use updated packages.


In [4]:
import numpy as np
import pandas as pd
import malariagen_data
import allel
import numba
import scipy
import plotly.express as px
from itertools import product, combinations
from scipy.special import comb
from scipy.spatial.distance import squareform
import matplotlib.pyplot as plt
import matplotlib
import plotly.graph_objects as go


In [5]:
af1 = malariagen_data.Af1(pre=True,
                          gcs_cache='/home/namulil/lstm_projects/funestus_llineup/gcs_cache',
                          results_cache='home/namulil/lstm_projects/funestus_llineup/results_cache'
                          )

In [6]:
# run this code to read in the additional sample metadata

additional_metadata = pd.read_csv('llineup_funestus_extra_metadata.csv')
additional_metadata['partner_sample_id'] = additional_metadata['partner_sample_id'].astype(str)

In [ ]:
additional_metadata

,IncAnalysis,RND,HSD,EA,HH,ID,HHID,HHID.rnd,Plate,Sample.identifier,...,Wave,HSD-name,Arm,Arm_ITT,Arm_PP,cohort_admin2_round,cohort_admin1_round,cohort_arm_round,cohort_armpp_round,cohort_subregions_round
0,1,1,80,4.0,14.0,1,108004014,8004014,4,6,...,3,Bugweri,OlysetPlus,PBO LLIN,PBO LLIN,Bugweri_Round-1,UG-E_round-1,OlysetPlus_round-1,PBO LLIN_round-1,East Central_round-1
1,1,1,80,6.0,88.0,3,108006088,8006088,4,92,...,3,Bugweri,OlysetPlus,PBO LLIN,PBO LLIN,Bugweri_Round-1,UG-E_round-1,OlysetPlus_round-1,PBO LLIN_round-1,East Central_round-1
2,1,1,71,8.0,112.0,3,107108112,7108112,8,25,...,2,Soroti Municipality,Olyset,Non-PBO LLIN,Non-PBO LLIN,Soroti_Round-1,UG-E_round-1,Olyset_round-1,Non-PBO LLIN_round-1,Mid-Eastern_round-1
3,1,1,71,8.0,112.0,6,107108112,7108112,8,61,...,2,Soroti Municipality,Olyset,Non-PBO LLIN,Non-PBO LLIN,Soroti_Round-1,UG-E_round-1,Olyset_round-1,Non-PBO LLIN_round-1,Mid-Eastern_round-1
4,1,1,71,8.0,112.0,7,107108112,7108112,8,73,...,2,Soroti Municipality,Olyset,Non-PBO LLIN,Non-PBO LLIN,Soroti_Round-1,UG-E_round-1,Olyset_round-1,Non-PBO LLIN_round-1,Mid-Eastern_round-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1145,1,4,28,9.0,48.0,4,402809048,2809048,71,87,...,4B,Busongora North,PermaNet-2,Non-PBO LLIN,Non-PBO LLIN,Kasese_Round-4,UG-W_round-4,PermaNet-2_round-4,Non-PBO LLIN_round-4,Mid-Western_round-4
1146,1,4,3,8.0,43.0,1,400308043,308043,71,4,...,4B,Bwamba,Olyset,Non-PBO LLIN,Non-PBO LLIN,Bundibugyo_Round-4,UG-W_round-4,Olyset_round-4,Non-PBO LLIN_round-4,Mid-Western_round-4
1147,1,4,3,8.0,43.0,4,400308043,308043,71,28,...,4B,Bwamba,Olyset,Non-PBO LLIN,Non-PBO LLIN,Bundibugyo_Round-4,UG-W_round-4,Olyset_round-4,Non-PBO LLIN_round-4,Mid-Western_round-4
1148,1,4,3,8.0,43.0,6,400308043,308043,71,40,...,4B,Bwamba,Olyset,Non-PBO LLIN,Non-PBO LLIN,Bundibugyo_Round-4,UG-W_round-4,Olyset_round-4,Non-PBO LLIN_round-4,Mid-Western_round-4


In [7]:
## checking RND data type in the extra meatadata before adding it to the af1 metadata
print(additional_metadata['RND'].dtype)


int64


In [8]:
### changing RND to a string before adding it to the af1 meta data
additional_metadata['RND'] = additional_metadata['RND'].astype(str)

In [9]:
## checking of conversion is succesful
print(additional_metadata['RND'].dtype)

object


In [10]:
# reading in the af1 meta dat
llineup_sample_set = '1288-VO-UG-DONNELLY-VMF00219'
df_samples = af1.sample_metadata(llineup_sample_set)

In [11]:
# run this code to add the additional metadata to the sample metadata available in the package
af1.add_extra_metadata(additional_metadata, on='partner_sample_id')

Load sample metadata: ⠸ (0:00:00.26) 

In [12]:
# run this code to retrieve a dataframe containing the combined sample metadata
llineup_sample_set = '1288-VO-UG-DONNELLY-VMF00219'
df_samples = af1.sample_metadata(llineup_sample_set)

In [ ]:
df_samples

,sample_id,partner_sample_id,contributor,country,location,year,month,latitude,longitude,sex_call,...,Wave,HSD-name,Arm,Arm_ITT,Arm_PP,cohort_admin2_round,cohort_admin1_round,cohort_arm_round,cohort_armpp_round,cohort_subregions_round
0,VBS64452-6645STDY13043152,108004014001,Martin Donnelly,Uganda,8004,2017,5,0.700,33.649,F,...,3,Bugweri,OlysetPlus,PBO LLIN,PBO LLIN,Bugweri_Round-1,UG-E_round-1,OlysetPlus_round-1,PBO LLIN_round-1,East Central_round-1
1,VBS64475-6645STDY13043175,108006088003,Martin Donnelly,Uganda,8006,2017,5,0.593,33.645,F,...,3,Bugweri,OlysetPlus,PBO LLIN,PBO LLIN,Bugweri_Round-1,UG-E_round-1,OlysetPlus_round-1,PBO LLIN_round-1,East Central_round-1
2,VBS64510-6645STDY13043210,107108112003,Martin Donnelly,Uganda,7108,2017,3,1.724,33.602,F,...,2,Soroti Municipality,Olyset,Non-PBO LLIN,Non-PBO LLIN,Soroti_Round-1,UG-E_round-1,Olyset_round-1,Non-PBO LLIN_round-1,Mid-Eastern_round-1
3,VBS64513-6645STDY13043213,107108112006,Martin Donnelly,Uganda,7108,2017,3,1.724,33.602,F,...,2,Soroti Municipality,Olyset,Non-PBO LLIN,Non-PBO LLIN,Soroti_Round-1,UG-E_round-1,Olyset_round-1,Non-PBO LLIN_round-1,Mid-Eastern_round-1
4,VBS64514-6645STDY13043214,107108112007,Martin Donnelly,Uganda,7108,2017,3,1.724,33.602,F,...,2,Soroti Municipality,Olyset,Non-PBO LLIN,Non-PBO LLIN,Soroti_Round-1,UG-E_round-1,Olyset_round-1,Non-PBO LLIN_round-1,Mid-Eastern_round-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1145,VBS67003-6645STDY13057647,402809048004,Martin Donnelly,Uganda,2809,2019,9,0.335,30.131,F,...,4B,Busongora North,PermaNet-2,Non-PBO LLIN,Non-PBO LLIN,Kasese_Round-4,UG-W_round-4,PermaNet-2_round-4,Non-PBO LLIN_round-4,Mid-Western_round-4
1146,VBS67004-6645STDY13057648,400308043001,Martin Donnelly,Uganda,308,2019,9,0.648,30.015,F,...,4B,Bwamba,Olyset,Non-PBO LLIN,Non-PBO LLIN,Bundibugyo_Round-4,UG-W_round-4,Olyset_round-4,Non-PBO LLIN_round-4,Mid-Western_round-4
1147,VBS67006-6645STDY13057650,400308043004,Martin Donnelly,Uganda,308,2019,9,0.648,30.015,F,...,4B,Bwamba,Olyset,Non-PBO LLIN,Non-PBO LLIN,Bundibugyo_Round-4,UG-W_round-4,Olyset_round-4,Non-PBO LLIN_round-4,Mid-Western_round-4
1148,VBS67007-6645STDY13057651,400308043006,Martin Donnelly,Uganda,308,2019,9,0.648,30.015,F,...,4B,Bwamba,Olyset,Non-PBO LLIN,Non-PBO LLIN,Bundibugyo_Round-4,UG-W_round-4,Olyset_round-4,Non-PBO LLIN_round-4,Mid-Western_round-4


In [ ]:
print('1288-VO-UG-DONNELLY-VMF00219')


1288-VO-UG-DONNELLY-VMF00219


In [ ]:
# keeping here to retest KDR in funestus code
#df = af1.aa_allele_frequencies(transcript='LOC125769886_t1', cohorts='admin1_year')
#af1.plot_frequencies_heatmap(df.query("max_af > 0.05"))


# checking to see if variant is under selection
#af1.plot_h12_gwss("3RL", window_size=1000, sample_query="country == 'Tanzania'", min_cohort_size=10)

In [13]:
## solving the numpy attricate error
import numpy as np
np.int = int

!pip install --upgrade networkx



In [14]:
import networkx as nx

defining regions

In [15]:
### describing all region sthat might needed for plotting 
gste2_snp= "2RL:76,406,705-76,407,655"
ace1_snp = "2RL:19,417,355-19,490,903"
cyp6aap_snp = "2RL:8,680,00-8,695,000"
cyp9k1_snp = "X:8,440,000-8,451,000"
eye_snp = "X:13,625,000-13,675,000"
gaba_snp = "3RL:13,537,199-13,612,792"
keap_snp = "2RL:14,868,000-14,883,000"
divers_stat = ":60,000,2RL000-80,000,000"
# ace1_transcript = "AGAP001356-RA"
vgsc_transcript = "AGAP004707-RD"

regions =[gste2_snp, ace1_snp, cyp6aap_snp, cyp9k1_snp, eye_snp, gaba_snp, keap_snp]

In [ ]:
### changing RND to a string 
#df_samples['RND'] = df_samples['RND'].astype(str)


In [ ]:
## checking if there is any missing value in RND
#print(df_samples['RND'].isnull().sum()) 

In [ ]:
#print(df_samples['RND'].dtype)


In [ ]:
### using code from Eric to check if i have any missing value 
#print((df_samples['RND']=="").sum())

In [ ]:
#rnd_dict = {'round_1':'RND == 1', 'round_2':'RND == 2', 'round_3':'RND == 3', 'round_4':'RND == 4', 'round_5':'RND == 5'}

#{f'round_{i}':f'RND == {i}' for i in np.arange(1,6)}

In [24]:
fig=af1.plot_njt(region = divers_stat,
             site_mask = "funestus",
             n_snps = 100000,
             sample_sets=llineup_sample_set,
             color = "RND",
             render_mode='svg',
             show= False
             )
# Save the fig to an SVG file
fig.write_image("/home/namulil/lstm_projects/funestus_llineup/notebooks/NJ trees/nj_RND2.svg", scale=2)

In [ ]:
fig2=af1.plot_njt(region = divers_stat,
             site_mask = "funestus",
             n_snps = 100000,
             sample_query="sample_id not in ['VBS65137-6645STDY13043869','VBS64809-6645STDY13043513','VBS65224-6645STDY13043958','VBS66747-6645STDY13045660', 'VBS65604-6645STDY13044458', 'VBS65603-6645STDY13044457','VBS66411-6645STDY13044218','VBS66999-6645STDY13057643','VBS65605-6645STDY13044459','VBS66747-6645STDY13045660','VBS66979-6645STDY13045894','VBS65539-6645STDY13044392','VBS65220-6645STDY13043954','VBS65525-6645STDY13044376','VBS65518-6645STDY13044369','VBS65519-6645STDY13044370','VBS65526-6645STDY13044377', 'VBS66973-6645STDY13045888', 'VBS65532-6645STDY13044383', 'VBS65555-6645STDY13044408', 'VBS65554-6645STDY13044407', 'VBS66968-6645STDY13045883', 'VBS66561-6645STDY13045471', 'VBS6490-6645STDY13043393','VBS64689-6645STDY13043392', 'VBS66991-6645STDY13057635', 'VBS66287-6645STDY13045195', 'VBS66986-6645STDY13057630','VBS66100-6645STDY13045007', 'VBS65528-6645STDY13044379','VBS65608-6645STDY13044463','VBS64906-6645STDY13043610','VBS66780-6645STDY13045693','VBS64905-6645STDY13043609','VBS64904-6645STDY130443608', 'VBS64919-6645STDY13043623', 'VBS66213-6645STDY13045121', 'VBS66586-6645STDY13045497', 'VBS64911-6645STDY13043615', 'VBS64910-6645STDY13043614', 'VBS66470-6645STDY13045377', 'VBS66474-6645STDY13045381', 'VBS65537-6645STDY13044390', 'VBS65543-6645STDY13044396']",
             color = "RND",
             sample_sets=llineup_sample_set,
             render_mode='svg',
             show=False
             )

# # Save the fig to an SVG file
fig2.write_image("/home/namulil/lstm_projects/funestus_llineup/notebooks/NJ trees/nj_Round.svg", scale=2)

In [ ]:
fig3=af1.plot_njt(region = divers_stat,
             site_mask = "funestus",
             n_snps = 100000,
             sample_query="sample_id not in ['VBS65137-6645STDY13043869','VBS64809-6645STDY13043513','VBS65224-6645STDY13043958','VBS66747-6645STDY13045660', 'VBS65604-6645STDY13044458', 'VBS65603-6645STDY13044457','VBS66411-6645STDY13044218','VBS66999-6645STDY13057643','VBS65605-6645STDY13044459','VBS66747-6645STDY13045660','VBS66979-6645STDY13045894','VBS65539-6645STDY13044392','VBS65220-6645STDY13043954','VBS65525-6645STDY13044376','VBS65518-6645STDY13044369','VBS65519-6645STDY13044370','VBS65526-6645STDY13044377', 'VBS66973-6645STDY13045888', 'VBS65532-6645STDY13044383', 'VBS65555-6645STDY13044408', 'VBS65554-6645STDY13044407', 'VBS66968-6645STDY13045883', 'VBS66561-6645STDY13045471', 'VBS6490-6645STDY13043393','VBS64689-6645STDY13043392', 'VBS66991-6645STDY13057635', 'VBS66287-6645STDY13045195', 'VBS66986-6645STDY13057630','VBS66100-6645STDY13045007', 'VBS65528-6645STDY13044379','VBS65608-6645STDY13044463','VBS64906-6645STDY13043610','VBS66780-6645STDY13045693','VBS64905-6645STDY13043609','VBS64904-6645STDY130443608', 'VBS64919-6645STDY13043623', 'VBS66213-6645STDY13045121', 'VBS66586-6645STDY13045497', 'VBS64911-6645STDY13043615', 'VBS64910-6645STDY13043614', 'VBS66470-6645STDY13045377', 'VBS66474-6645STDY13045381', 'VBS65537-6645STDY13044390', 'VBS65543-6645STDY13044396']",
             color = "admin1_iso",
             sample_sets=llineup_sample_set,
             show=False
             )

# Save the fig to an SVG file
fig3.write_image("/home/namulil/lstm_projects/funestus_llineup/notebooks/NJ trees/nj_region.svg", scale=2)

In [ ]:
af1.plot_njt(region = divers_stat,
             site_mask = "funestus",
             n_snps = 100000,
             sample_query="sample_id not in ['VBS65137-6645STDY13043869','VBS64809-6645STDY13043513','VBS65224-6645STDY13043958','VBS66747-6645STDY13045660', 'VBS65604-6645STDY13044458', 'VBS65603-6645STDY13044457','VBS66411-6645STDY13044218','VBS66999-6645STDY13057643','VBS65605-6645STDY13044459','VBS66747-6645STDY13045660','VBS66979-6645STDY13045894','VBS65539-6645STDY13044392','VBS65220-6645STDY13043954','VBS65525-6645STDY13044376','VBS65518-6645STDY13044369','VBS65519-6645STDY13044370','VBS65526-6645STDY13044377', 'VBS66973-6645STDY13045888', 'VBS65532-6645STDY13044383', 'VBS65555-6645STDY13044408', 'VBS65554-6645STDY13044407', 'VBS66968-6645STDY13045883', 'VBS66561-6645STDY13045471', 'VBS6490-6645STDY13043393','VBS64689-6645STDY13043392', 'VBS66991-6645STDY13057635', 'VBS66287-6645STDY13045195', 'VBS66986-6645STDY13057630','VBS66100-6645STDY13045007', 'VBS65528-6645STDY13044379','VBS65608-6645STDY13044463','VBS64906-6645STDY13043610','VBS66780-6645STDY13045693','VBS64905-6645STDY13043609','VBS64904-6645STDY130443608', 'VBS64919-6645STDY13043623', 'VBS66213-6645STDY13045121', 'VBS66586-6645STDY13045497', 'VBS64911-6645STDY13043615', 'VBS64910-6645STDY13043614', 'VBS66470-6645STDY13045377', 'VBS66474-6645STDY13045381', 'VBS65537-6645STDY13044390', 'VBS65543-6645STDY13044396']",
             color = "cohort_admin1_year",
             sample_sets=llineup_sample_set
             )

Troublehooting with Eric why RND returns a list with missing values

In [ ]:
df_samples['RND']

0       1
1       1
2       1
3       1
4       1
       ..
1145    4
1146    4
1147    4
1148    4
1149    4
Name: RND, Length: 1150, dtype: object

In [ ]:
np.unique(df_samples['RND'], return_counts = True)

(array(['1', '2', '3', '4', '5'], dtype=object),
 array([ 39,  58,  66, 576, 411]))